In [2]:
import pygame
from gamemaker import * #UI
from time import sleep
import random
from keyboard import is_pressed #키보드
from pingpongthread import PingPongThread #핑퐁
import qrcode #QR 코드
import pyttsx3
import cv2

import numpy as np
from pyzbar.pyzbar import decode

In [3]:
p = PingPongThread(number=3)
# light = 1번 / dot = 2번 / gas(큐브이노)+ 자이로 = 3번
p.start()
p.wait_until_full_connect() #핑퐁 로봇 블루투스 연결

Bluetooth is available.
BLE connected.
Write data: FF FF 00 FF 30 00 AD 00 0B 0A 00
Connected robots: 2
Connected robots: 3
Fully connected.


In [4]:
p.receive_sensor_data(1, "periodic", 0.1)
p.receive_sensor_data(3, "periodic", 0.1)

Write data: FF FF FF FF 00 C8 B8 00 0B 0A 01
Write data: FF FF FF FF 00 C8 B8 00 0B 0A 01


In [60]:
def check():
    cap = cv2.VideoCapture(0)  # 기본 웹캠
    data = ""
    if not cap.isOpened():
        print("카메라를 열 수 없습니다.")
        self.qr_text = "카메라 오류"
    recognized = False
    
    while not recognized:
        ret, frame = cap.read()
        if not ret:
            print("프레임을 읽을 수 없습니다.")
            break
    
        decoded_objects = decode(frame)
        for obj in decoded_objects:
            data = obj.data.decode('utf-8')
            print("디코딩된 데이터:", data)
            # self.qr_text = data  # QR 결과 저장
            recognized = True  # 루프 종료 조건
            
            # 사각형 그리기 (선택 사항)
            points = obj.polygon
            if len(points) == 4:
                pts = [(point.x, point.y) for point in points]
                cv2.polylines(frame, [np.array(pts)], True, (0, 255, 0), 2)
                
        cv2.imshow("QR Code Scanner", frame)
    
        # 사용자가 'q' 누르거나 결과 인식되면 종료
        if cv2.waitKey(1) & 0xFF == ord('q') or recognized:
            break
    
    cap.release()
    cv2.destroyAllWindows()
    if data == "none":
        data = "승인되지 않았습니다."
    return data

In [50]:
cap.release()
cv2.destroyAllWindows()

In [62]:
pygame.init()
pygame.display.set_caption("스마트 안전 시스템")
display = pygame.display.set_mode((500, 700))

g = GAMEMAKER(display)

#임계값 설정
GAS_THRESHOLD = 40 # 가스센서 임계값
GYRO_THRESHOLD = 20 # 자이로 센서 임계값
LIGHT_THRESHOLD = 130 #조도 센서 임계값 

# 색상 정의
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 200, 0)
YELLOW = (225, 225, 0)

# TTS 설정
tts_engine = pyttsx3.init()

# 메시지 로그 및 종료 버튼
message_log = []
# 버튼 위치 및 색상 정의
btn_exit_rect = pygame.Rect(200, 420, 100, 40)
# QR 인식 버튼
btn_qr_rect = pygame.Rect(200, 360, 100, 40)

# 조끼 클래스
class Vest:
    def __init__(self):
        self.gas = 0
        self.qr_text = "없음"
        self.gas_full = False
    
    def check_gas(self):
        gas_value = p.get_arduino_analog_value(3, "A0")
        self.gas = gas_value  # 실시간으로 읽어온 값 저장
        if gas_value > GAS_THRESHOLD:
            if not self.gas_full:
                tts_engine.say("가스가 유출되었습니다. 대피하세요.")
                tts_engine.runAndWait()
            self.gas_full = True
        else:
            self.gas_full = False

cap.release()
cv2.destroyAllWindows()
# 헬멧 클래스
class Helmet:
    def __init__(self):
        self.light = 0
        self.gyro = (0, 0, 0)
        self.fallen = False

    def light_sensor(self):
        light_value = p.get_current_extensionSensor(1, "Light")
        self.light = light_value  # 값 저장
        if light_value < LIGHT_THRESHOLD:
            pattern = [[1]*8 for _ in range(8)]
            p.LED_matrix_write_picture(2, pattern)
        else:
            blank = [[0]*8 for _ in range(8)]
            p.LED_matrix_write_picture(2, blank)

    def gyro_sensor(self):
        x,y,z = p.get_current_gyro(3)
        self.gyro = (x,y,z)
        if abs(x) > GYRO_THRESHOLD or abs(y) > GYRO_THRESHOLD or abs(z) > GYRO_THRESHOLD:
            if not self.fallen:
                tts_engine.say("넘어짐이 감지되었습니다. 구조요청 대피해보세요.")
                tts_engine.runAndWait()
                self.fallen = True
        else:
            self.fallen = False
            
# --- UI 표시 함수 ---
def draw_ui(vest, helmet):
    display.fill(WHITE)

    # 조끼부분 UI 출력
    
    if vest.gas > GAS_THRESHOLD:
        gas_color = RED
    else:
        gas_color = BLACK

    g.draw_text(g.small_font,f"가스 수치: {vest.gas}", (250, 120),  gas_color)

    # Helmet 부분 UI 출력
    # 라이트 UI 출력
    g.draw_text(g.small_font,f"조도 수치: {helmet.light}", (250, 160), BLACK)

    if helmet.light < LIGHT_THRESHOLD:
        g.draw_text(g.small_font,f"라이트 켜짐", (250, 190), YELLOW)
    else:
        g.draw_text(g.small_font,f" ", (250, 190), WHITE)

    # 자이로 UI 출력
    x, y, z = helmet.gyro
    g.draw_text(g.small_font,f"자이로: x={x:.2f}, y={y:.2f}, z={z:.2f}", (250, 270), BLACK)
    
    if helmet.fallen:
        g.draw_text(g.small_font,"넘어짐 감지됨!", (250, 310), RED)
    else:
        g.draw_text(g.small_font,"정상 상태", (250, 240), GREEN)

    # 종료 버튼 추가
    pygame.draw.rect(display, RED, btn_exit_rect)
    
    g.draw_text(g.small_font,"종료", (btn_exit_rect.x+50, btn_exit_rect.y+20), WHITE)

    #     # QR 인식 버튼 추가
    pygame.draw.rect(display, GREEN, btn_qr_rect)
    g.draw_text(g.small_font, "QR 인식", (btn_qr_rect.x + 50, btn_qr_rect.y + 20), WHITE)

    pygame.display.update()

    

# --- 메인 루프 ---
def main():
    vest = Vest()
    helmet = Helmet()
    res = ""
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if btn_exit_rect.collidepoint(event.pos):
                    running = False
                elif btn_qr_rect.collidepoint(event.pos):
                    res = check()

        p.clear_output()
        vest.check_gas()
        helmet.light_sensor()
        helmet.gyro_sensor()
        draw_ui(vest, helmet)
        g.draw_text(g.small_font, f"QR 인식 : {res}", (btn_exit_rect.x+50, btn_exit_rect.y+80), BLACK)
        g.tick(1)

    pygame.quit()

if __name__ == "__main__":
    main()
    

Write data: FF FF FF 01 30 E2 A2 00 12 70 00 00 00 00 00 00 00 00


In [59]:
pygame.quit()